# IMDClient package architecture

### 1. Use imdclient outside of MDAnalysis
### 2. Client is "batteries included"
### 3. Strive for full MDAnalysis compatibility



![image](imd-mda_1.jpg)

## IMDClient API Methods

### 1. `get_imdsessioninfo()`
### 2. `get_imdframe()`
### 3. `stop()` *(handled by context manager)*

In [ ]:
import imdclient

with imdclient.IMDClient("localhost", 8889, n_atoms=50786) as client:
    info = client.get_imdsessioninfo()
    print(info)

    frame = client.get_imdframe()
    print(f"Simulation integration step: {frame.step}")
    print(f"Simulation time (fs): {frame.time}")
    print(f"First atom's position (angstroms): {frame.positions[0]}")

## Iterating through a trajectory

### Call `get_imdframe()` until EOFError is raised
### Slicing isn't possible

In [ ]:
import imdclient

with imdclient.IMDClient("localhost", 8889, n_atoms=50786) as client:
    while True:
        try:
            frame = client.get_imdframe()
            # Do something with the frame
        except EOFError:
            break

## Options

### Configurable timeout for high latency
### Configurable buffer size for fast simulations

In [ ]:
import imdclient

client = imdclient.IMDClient("localhost", 8889,
                              n_atoms=50786, 
                              # Wait up to 10 seconds for a simulation frame
                              timeout=10,
                              # 1 MB
                              buffer_size=1024 ** 2)

## Software architecture

![img2](imd-mda_2.jpg)

![img4](imd-mda_3.jpg)

## Automatic pausing and resuming


In [12]:
import imdclient

# 2MB Buffer
with imdclient.IMDClient("localhost", 8889, n_atoms=50786, buffer_size=2 * 1024**2) as client:
    while True:
        try:
            frame = client.get_imdframe()
            print(f"The position of the first atom is: {frame.positions[0]}", end="\r")
        except EOFError:
            break

## Reader wraps client, handles its limitations

### The imdclient can't be sliced, so the reader can't either
### The imdclient only moves forward, so the reader does too

## API interactions

![img4](imd-mda_4.jpg)

In [10]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

for ts in u.trajectory[:]:
    pass

try:
    for ts in u.trajectory[:]:
        pass
except RuntimeError:
    print("A stream is not a file!")

A stream is not a file!


### Compatible ✅
```python
for ts in u.trajectory[::10]:
    pass

for ts in u.trajectory:
    pass

for ts in u.trajectory[:]:
    pass
```

### Incompatible (raises `RuntimeError`) ❌

```python
for ts in u.trajectory[:10]:
    pass

for ts in u.trajectory[10:]:
    pass

for ts in u.trajectory[::-1]:
    pass

len(u.trajectory)

u.trajectory.n_frames
```


## The client works out of the box with some MDAnalysis analysis classes

### Caveats:

- The analysis class must be able to handle a trajectory without a known length
- The analysis base class is patched automatically on importing imdclient

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

imd_rmsf = RMSF(u.atoms).run()

print(imd_rmsf.rmsf)

## Running multiple analysis classes on the same stream


In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

r1 = RMSF(u.atoms)
r2 = RMSF(u.atoms)
imdclient.StackableAnalysis(u.trajectory, [r1, r2]).run()